In [ ]:
from pathlib import Path
import cv2
from PIL import Image
import pytesseract
from corpus_unpdf import get_img

In [ ]:
ROI = "temp/roi.png"
HEADER_TOP = 350
WIDTH_HEIGHT = (50,10)
COLOR_GREEN = (36, 255, 12)
LINE_INDICATOR_HEIGHT = 20
LINE_INDICATOR_WIDTH = 300

In [ ]:
PAGE_NUM = 2
im = get_img(Path().cwd() / "data" / "data1.pdf", pagenum=PAGE_NUM)

In [ ]:
import numpy
opencvImage = cv2.cvtColor(numpy.array(im), cv2.COLOR_RGB2BGR)
base_image = opencvImage.copy()
im_h, im_w, im_d = opencvImage.shape

In [ ]:
gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, WIDTH_HEIGHT)
dilate = cv2.dilate(thresh, kernel, iterations=1)
cv2.imwrite("temp/sample_dilated.png", dilate)

In [ ]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])

In [ ]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if w > LINE_INDICATOR_WIDTH and h < LINE_INDICATOR_HEIGHT: # spot footnote indicator, see related kernel (50, 10)
        print(f"{x=}, {y=}, {w=}, {h=}")
        cv2.rectangle(opencvImage, (x,y), (x+w, y+h), COLOR_GREEN, 2)
        roi = base_image[HEADER_TOP : y + h, 0 : x + im_w]
cv2.imwrite("temp/sample_boxes.png", opencvImage)

In [ ]:
from PIL import Image
try: # roi found
    cv2.imwrite(ROI, roi)
    ocr_result = pytesseract.image_to_string(roi)
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=100.0, save_all=True
    )	
    print("Reveal ROI")    

except NameError: # no roi, just remove header
    cv2.imwrite(ROI, base_image)
    ocr_result = pytesseract.image_to_string(base_image[HEADER_TOP:im_h, 0:im_w])
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=300.0, save_all=True
    )	
    print("Reveal base")


In [ ]:
ocr_result

In [ ]:
import pdfplumber
pdf = pdfplumber.open("temp/temp.pdf")

In [ ]:
pdf.pages[0].extract_text()